# Fine-tuning a bidirectional pretrained language model

## the task

The task is to assign a coarse semantic category to word sense definitions, from wiktionary, for French.

The semantic categories are "supersenses".

The aproach will be to encode the definition using a BERT-like pretrained language model (pretrained for French), take the bert output vector for the [CLS] token, and run a linear transformation into the set of supersenses.


## the data

In [ ]:
import pandas as pd
import os

if not os.path.exists('./gold_supersenses_wiktionary_defs.tsv'):
  !pip install wget
  import wget

  url = "https://www.linguist.univ-paris-diderot.fr/~mcandito/divers/gold_supersenses_wiktionary_defs.tsv.gz"
  output_path = './gold_supersenses_wiktionary_defs.tsv.gz'

  if not os.path.exists(output_path):
    print('Downloading dataset')
    # Added --no-check-certificate to bypass SSL verification
    os.system(f"wget --no-check-certificate {url} -O {output_path}")
    os.system(f"gunzip -f {output_path}")



  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=b25667e592f4b9f82b22f6ec3a4d2270e0693facdf6045aafd4be21ab2d3a712
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [ ]:
df = pd.read_csv('./gold_supersenses_wiktionary_defs.tsv', sep='\t')
df.drop(columns = ['ann_stage', 'labels', 'synonyms', 'nb_occ_lemma_frsemcor'], inplace = True)
df.drop( df[ (df['set'] != 'train') & (df['set'] != 'freq-test') & (df['set'] != 'freq-dev')].index, inplace=True)
display(df.head())

/tmp/ipython-input-4152270429.py:1: DtypeWarning: Columns (8,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./gold_supersenses_wiktionary_defs.tsv', sep='\t')


sense_id            entry_id       lemma     supersense  \
42     ws_4_médecine__nom__1    médecine__nom__1    médecine      substance   
56        ws_5_lapin__nom__1       lapin__nom__1       lapin         person   
172        ws_7_lien__nom__1        lien__nom__1        lien  communication   
291     ws_2_colonel__nom__1     colonel__nom__1     colonel         person   
329  ws_5_communauté__nom__1  communauté__nom__1  communauté    institution   

           set                                         definition  \
42    freq-dev  Remède, sous forme liquide ou solide qu’on pre...   
56    freq-dev  Nom hypocoristique donné à sa ou son partenair...   
172   freq-dev  Moyen hypertexte de transfert vers une source ...   
291   freq-dev               Fonctionnaire titulaire de ce grade.   
329  freq-test       Maison religieuse où l’on vit en communauté.   

                                             example_1  \
42   {{exemple|Halpersohn, qui passa, pendant cinq ...   
56                                   Je t’aime, lapin.   
172  Ajouter un lien. Faire la référence à un autre...   
291  Un colonel des sapeurs-pompiers, des Douanes, ...   
329                        Le jardin de la communauté.   

                                             example_2 example_3 example_4  \
42   {{exemple|Base d’une sorte de médecine coraniq...       NaN       NaN   
56                 Ne rentre pas trop tard, mon lapin.       NaN       NaN   
172  Utiliser, suivre un lien automatique par cliquer.       NaN       NaN   
291                                                NaN       NaN       NaN   
329                Nous visitâmes toute la communauté.       NaN       NaN   

     ... example_14 example_15 example_16 example_17 example_18 example_19  \
42   ...        NaN        NaN        NaN        NaN        NaN        NaN   
56   ...        NaN        NaN        NaN        NaN        NaN        NaN   
172  ...        NaN        NaN        NaN        NaN        NaN        NaN   
291  ...        NaN        NaN        NaN        NaN        NaN        NaN   
329  ...        NaN        NaN        NaN        NaN        NaN        NaN   

    example_20 example_21 example_22 example_23  
42         NaN        NaN        NaN        NaN  
56         NaN        NaN        NaN        NaN  
172        NaN        NaN        NaN        NaN  
291        NaN        NaN        NaN        NaN  
329        NaN        NaN        NaN        NaN  

[5 rows x 29 columns]

In [ ]:
# keep only the rows with supersenses having more 150 instances
labels = [ x for (x,y) in df['supersense'].value_counts().items() if y > 150 ]
print(labels)
num_labels = len(labels)
df = df[df["supersense"].isin(labels)]

['act', 'artifact', 'cognition', 'person', 'attribute', 'state', 'event', 'object', 'substance', 'institution', 'animal', 'plant', 'body', 'food', 'quantity', 'possession', 'feeling', 'act*cognition', 'time', 'communication', 'artifact*cognition', 'phenomenon']


In [ ]:
for split in ['train', 'freq-test', 'freq-dev']:
  print(len(df[df['set'] == split]))

9935
1310
1538


In [ ]:
full_df = df

# while coding and testing : use a fraction of the data
small_df = df.sample(n=1000)
for split in ['train', 'freq-test', 'freq-dev']:
  print(len(small_df[small_df['set'] == split]))


757
97
146


# Loading a Huggingface tokenizer and model

## Huggingface : Browsing the available models

Huggingface is a hub for a huge number of transformer-based models, either pre-trained models (generally to be used in additional training for downstream tasks) or already fine-tuned models.

Models in Huggingface are identified by a name. E.g. "bert-base-multilingual-cased". Cf. the list of models https://huggingface.co/models?sort=downloads

Each model is tagged with several tags, among which is the main (pre)training task. In the models' page, you can filter by tag (left panel).

In [ ]:
import torch

# import (after potential installation) of the transformers module
try:
  from transformers import AutoModel, AutoTokenizer, AutoConfig
except ImportError:
  !pip install transformers
  from transformers import AutoModel, AutoTokenizer, AutoConfig

## The tokenizer


Each model is associated with a specific tokenizer: the one used for tokenizing the (pre-)training corpus.

Given a model name, the AutoTokenizer / AutoModel / AutoConfig classes can "guess" the exact python class for the corresponding tokenizer / model / config.

For instance the **'bert-base-multilingual-cased'** name corresponds to the BertModel, BertTokenizerFast and BertConfig classes.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("almanach/camembertav2-base")
print(type(tokenizer))

config = AutoConfig.from_pretrained("almanach/camembertav2-base")
print(config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

<class 'transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast'>


config.json: 0.00B [00:00, ?B/s]

DebertaV2Config {
  "architectures": [
    "DebertaV2Model"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 1,
  "conv_act": "gelu",
  "conv_kernel_size": 0,
  "embedding_size": 768,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "legacy": true,
  "max_position_embeddings": 1024,
  "max_relative_positions": -1,
  "model_name": "camembertav2-base",
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torch_dtype": "float32",
  "transformers_version": "4.55.4",
  "type_vocab_size": 0,
  "vocab_size": 32768
}



In [ ]:
sent = "Aujourd'hui, j'essaie de comprendre les intringuants transformers."

# "encode" method: tokenization + conversion to token ids :
print("1:", tokenizer.encode(sent))
# readable form
print("2:", tokenizer.convert_ids_to_tokens(tokenizer.encode(sent)))
print()

# without special symbols:
print("3:", tokenizer.encode(sent, add_special_tokens=False))
print("4:", tokenizer.convert_ids_to_tokens(tokenizer.encode(sent, add_special_tokens=False)))
print()

# equivalent to:
print("5:", tokenizer.tokenize(sent))
print("6:", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)))
print()

1: [1, 8682, 14, 5586, 19, 5192, 15540, 4702, 7521, 4733, 17197, 4985, 15857, 3206, 13046, 3206, 21, 2]
2: ['[CLS]', 'Aujourd', "'", 'hui', ',', "j'", 'essaie', 'de', 'comprendre', 'les', 'intr', '##ing', '##uant', '##s', 'transformer', '##s', '.', '[SEP]']

3: [8682, 14, 5586, 19, 5192, 15540, 4702, 7521, 4733, 17197, 4985, 15857, 3206, 13046, 3206, 21]
4: ['Aujourd', "'", 'hui', ',', "j'", 'essaie', 'de', 'comprendre', 'les', 'intr', '##ing', '##uant', '##s', 'transformer', '##s', '.']

5: ['Aujourd', "'", 'hui', ',', "j'", 'essaie', 'de', 'comprendre', 'les', 'intr', '##ing', '##uant', '##s', 'transformer', '##s', '.']
6: [8682, 14, 5586, 19, 5192, 15540, 4702, 7521, 4733, 17197, 4985, 15857, 3206, 13046, 3206, 21]



In [ ]:
texts = ["eoari8 é§ êêçth et aux avec ceux-ci, sont-ils et est-t-elle?",
         "J'essaie de comprendre les transformers.",
         "Nous comprenions bien le cours.",
         "Le code comprend des erreurs.",
         "Ils ont très bien compris le nouveau cours!"]

encodings = tokenizer(texts, max_length=10, truncation=True, padding=True)
for key, l in encodings.items():
    print(key+":\n"+"\n".join([str(x) for x in l]))


# in a more readable format:
for token_ids in encodings['input_ids']:
    print(tokenizer.convert_ids_to_tokens(token_ids))

# use return_tensors = 'pt' for pytorch tensors
tensor_encodings = tokenizer(texts, max_length=10, truncation=True, padding=True, return_tensors="pt")
print(tensor_encodings['input_ids'].shape)


# NB:the models are trained with padding tokens appearing at the end of sequences only
# hence applying the models to sequences **starting** with padding tokens
# will result in bad-quality contextual embeddings

for key, value in tensor_encodings.items():
    print(f"{key}: {value.numpy().tolist()}")



input_ids:
[1, 76, 20730, 4737, 4635, 186, 121, 187, 3300, 2]
[1, 5655, 15540, 4702, 7521, 4733, 13046, 3206, 21, 2]
[1, 5361, 5966, 5406, 4999, 4722, 5667, 21, 2, 0]
[1, 4833, 7496, 8421, 4730, 12241, 21, 2, 0, 0]
[1, 6063, 4982, 5087, 4999, 7224, 4722, 5716, 5667, 2]
attention_mask:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['[CLS]', 'e', '##oa', '##ri', '##8', 'é', '§', 'ê', '##ê', '[SEP]']
['[CLS]', "J'", 'essaie', 'de', 'comprendre', 'les', 'transformer', '##s', '.', '[SEP]']
['[CLS]', 'Nous', 'compren', '##ions', 'bien', 'le', 'cours', '.', '[SEP]', '[PAD]']
['[CLS]', 'Le', 'code', 'comprend', 'des', 'erreurs', '.', '[SEP]', '[PAD]', '[PAD]']
['[CLS]', 'Ils', 'ont', 'très', 'bien', 'compris', 'le', 'nouveau', 'cours', '[SEP]']
torch.Size([5, 10])
input_ids: [[1, 76, 20730, 4737, 4635, 186, 121, 187, 3300, 2], [1, 5655, 15540, 4702, 7521, 4733, 13046, 3206, 21, 2], [1, 

## The pretrained model

In [ ]:
# loading the model
model = AutoModel.from_pretrained("almanach/camembertav2-base")
print(type(model))
# see doc https://huggingface.co/docs/transformers/main/en/model_doc/deberta-v2



model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

<class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2Model'>


In [ ]:
model.eval()

DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(32768, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, i

In [ ]:
# running the model
output = model(input_ids = tensor_encodings['input_ids'],
               attention_mask = tensor_encodings['attention_mask'],
               return_dict=True)
# Note : also possible to use output_attentions=True and output_hidden_states=True
print("\nKeys of the output dict after forward propag:\n", output.keys())
print("\nShape at the last layer:\n", output.last_hidden_state.shape) # batch size, seq length 10, hidden vector size 768





Keys of the output dict after forward propag:
 odict_keys(['last_hidden_state'])

Shape at the last layer:
 torch.Size([5, 10, 768])


# TODO: build and train a network fine-tuning the deBERTa model

- tokenize the definitions, truncate and pad them
- tip : prepend the lemma + ':' to the sense definition, to benefit from the embedding of the lemma learnt in the pretrained language model
- turn the labels (column "supersense") into label ids
- build a TensorDataset and a Dataloader for the three splits of the data (train/dev/test)
  * each tensordataset has to contain the input_ids, the attention_masks, and the labels

- declare a network whose forward will
  * take as input a tensor of padded wiktionary definitions
    * => shape (batch_size, max_len)
  * runs the camembertav2 model on the batch to get contextualized vector
    * => (batch_size, max_len, 768)
  * keeps the output contextualized vectors of the [CLS] tokens
    * => (batch_size, 768)
  * applies a dropout (rate : 0.1)
  * and a Linear layer
    * => (batch_size, num_labels)
  * NB: in the __init__, the layers of parameters have to be declared using .to(device) (see below), in order to have them directly on the right device (cpu or gpu)

- implement training of the network
  * NB: while debugging, freeze the parameters of the deberta model, to speed up!
  * NB: when looping over batches, move the batch tensor to the target device (.to(device))

  

In [ ]:

class SupersenseClassifier(nn.Module):

    def __init__(self, num_labels, device, bert_model, freeze_bert = False):
        super(SupersenseClassifier, self).__init__()

        self.freeze_plm = freeze_plm

        self.device = device

        # the full *BERT*-like model (in our case camemBERTav2)
        # the .to(device) triggers the copy towards the relevant device
        # (possibly a GPU)
        self.plm_layer = bert_model.to(device)
        # config will allow to get the hidden vectors' size
        self.hidden_size = bert_config.hidden_size

        # to freeze bert model parameters : don't compute their gradients
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        print("Classifier instance, freeze_bert = %s" % str(freeze_bert))

        # TODO
        # ...
        # NB : use .to(device) for all the declared layers

    def forward(self, input_ids, attention_masks):
        # TODO
        # ...

SyntaxError: incomplete input (ipython-input-810756139.py, line 29)